Random Forest is a popular ensemble learning method used for classification, regression, and other tasks. It combines the predictions of multiple decision trees to achieve greater accuracy and robustness compared to a single decision tree.

1. Model Definition
2. Training and testing on Diabetes Dataset
3. Metrics

## Importing Libraries

In [3]:
import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

import pandas as pd

from collections import Counter
from decision_tree import DecisionTree

## Model Definition

Using previously written Decision tree

## Bootstrap Sampling

Creates a random sample of data with replacement (bootstrap sample) for training each decision tree.

## Most Common Label

Determines the most frequent label in an array (used for majority voting in classification).

In [4]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]


def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

In [5]:
class RandomForest:

    def __init__(self, n_trees=10, min_samples_split=2,
                 max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(min_samples_split=self.min_samples_split,
                                max_depth=self.max_depth, n_feats=self.n_feats)
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

For each tree:
* Create a bootstrap sample of the data.
* Train a DecisionTree on the sample.
* Append the trained tree to self.trees.

In [6]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

Computes the proportion of correct predictions out of the total predictions.

## Training and testing on Diabetes Dataset

In [7]:
cols = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness",
        "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]
url = "https://gist.githubusercontent.com/ktisha/c21e73a1bd1700294ef790c56c8aec1f/raw/819b69b5736821ccee93d05b51de0510bea00294/pima-indians-diabetes.csv"
diabetes_data = pd.read_csv(url, skiprows=9, header=None, names=cols)

Loads the Pima Indians Diabetes dataset for training and evaluation.

In [8]:
diabetes_data.shape


(768, 9)

In [9]:
X = diabetes_data[cols[:-1]].values
y = diabetes_data[cols[-1]].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)


Splits the dataset into training and test sets.

In [11]:
rnd_clf = RandomForest(max_depth=10, n_trees=10, n_feats=6)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)

* Initialize a RandomForest classifier with 10 trees and a maximum depth of 10.
* Train the classifier on the training data.
* Predict labels for the test data.
* Compute and print the accuracy of the predictions.

## Metrics

In [12]:
print(f"Accuracy: {accuracy(y_test, y_pred)}")


Accuracy: 0.7662337662337663


In [13]:
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1-Score: {f1_score(y_test, y_pred)}")

Precision: 0.6451612903225806
Recall: 0.7407407407407407
F1-Score: 0.6896551724137931
